In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torchvision import transforms
from torch.nn import init
from collections import OrderedDict
import torchvision.datasets as datasets
from torchvision.models.segmentation import segmentation
import torchvision.transforms.functional as FT
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from torch.utils.data import Dataset, DataLoader
import multiprocessing
from datetime import datetime
from PIL import ImageFilter
from PIL import Image
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Set device to GPU_indx if GPU is avaliable
GPU_indx = 0
device = torch.device(GPU_indx if torch.cuda.is_available() else 'cpu')

In [ ]:
cd drive/MyDrive/AI3/

/content/drive/MyDrive/AI3


In [ ]:
Img_c = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ColorJitter(brightness=[0.4,1.8], contrast=[0.5,1.5], saturation=[0,2.0], hue=[-0.1,0.1]), #, contrast=0.5, saturation=0.4, hue=0.1 brightness=[0.4,1.7], contrast=[0.5,1.5] saturation=[0,2.0] hue=[-0.2,0.2]
        transforms.ToTensor(),])

transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomCrop((280,280)), 
            # transforms.RandomPerspective(distortion_scale=0.6, p=1.0),
            transforms.ToTensor(),
            transforms.RandomErasing(p=0.5, scale=(0.04, 0.05)),])

default = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(p=1),    
            transforms.ToTensor()])

In [ ]:
class DriveData(Dataset):
    def __init__(self, trn_val_tst = 0, transform=None):
        if (trn_val_tst==0):
            data = np.load("mapil_3/train.npz", mmap_mode='r')
        elif (trn_val_tst==1):
            data = np.load("mapil_3/val.npz", mmap_mode='r')
        # elif (trn_val_tst==2):
        #     data = np.load("test.npz", mmap_mode='r')
        
        self.images = data['arr_0']
        self.images = torch.tensor(self.images)
        self.images = self.images.permute(0,3,1,2)

        # Conditional statement to detect labels in dataset
        if (trn_val_tst==0 or trn_val_tst==1): 
            self.labels = data['arr_1']
            self.labels = torch.tensor(self.labels)
        else:
            self.labels = None 

        self.trn_val_tst = trn_val_tst
        self.transform = transform

    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        # Setting random seed
        seed = random.randint(0,2**32)
        
        # Loading images and labels for transformation
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = self.images[idx,:]
        if (self.labels is not None):
            labels = self.labels[idx,:]
        
        
        if (self.transform is not None):
            if (self.trn_val_tst == 0):
              sample = Img_c(sample)
              # Setting same seed for the transformation to apply same transformation for the images and labels
              torch.manual_seed(seed)
              sample = self.transform(sample)
            else:
              # Setting same seed for the transformation to apply same transformation for the images and labels
              torch.manual_seed(seed)
              sample = self.transform(sample)
        
        if (self.labels is not None):
            # Setting same seed for the transformation to apply same transformation for the images and labels
            torch.manual_seed(seed)
            labels = torch.round(self.transform(labels)*255)
            return sample, labels
        return sample

In [ ]:
# Reading the dataset and storing them to a variable
train_set = DriveData(trn_val_tst=0, transform=transform) 
val_set = DriveData(trn_val_tst=1, transform=default) 
# test_set = DriveData(trn_val_tst=2, transform=default) 

In [ ]:
# Setting batch size, multiprocessing modules and loading them as dataloader
batch_size = 3
n_workers = 2
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=n_workers)
validloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                         shuffle=False, num_workers=n_workers)
# testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
#                                          shuffle=False, num_workers=n_workers)

In [ ]:
# 0 is NOTHING
# 1 is PEDESTRIAN
# 2 is CAR



for i, (x, y) in enumerate(trainloader):
    if (i<5):
        image = x[0]
        label = y[0]
        plt.figure(figsize=(10,10))
        plt.imshow(image.permute(1, 2, 0))
        plt.figure(figsize=(10,10))
        plt.imshow(label.permute(1, 2, 0).squeeze(2))

In [ ]:
print(trainloader.__len__())

2000


In [ ]:
# Initialising the model, pretrained weight is currently not available for fcn_resnet50
# net = segmentation.fcn_resnet50(pretrained=False, progress=True, num_classes=3).to(device)
net = segmentation.lraspp_mobilenet_v3_large(pretrained=False, progress=False, num_classes=3).to(device)
print(net)

#Pass our network parameters to the optimiser set our lr as the learning_rate
optimizer = optim.AdamW(net.parameters(), lr = 1e-4)

#Define a Cross Entropy Loss
Loss_fun = nn.CrossEntropyLoss()

#epochs
num_epochs=20

#where to load/save the dataset from 
data_set_root = "data"
Start_From_Checkpoint = False
save_dir = 'Models'
model_name = 'FCN' + str(datetime.now())

In [ ]:
#Create Save Path from save_dir and model_name, we will save and load our checkpoint here
Save_Path = os.path.join(save_dir, model_name + ".pt")

#Create the save directory if it does note exist
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

In [ ]:
# loading pretrained model -- USED FOR CONTINUATION OF TRAINING MODEL
Pretrained_Path = "./Models/bdd_city_320_w_mapil_2.pt"
check_point = torch.load(Pretrained_Path)
net.load_state_dict(check_point['model_state_dict'])
optimizer.load_state_dict(check_point['optimizer_state_dict'])

In [ ]:
#This function should perform a single training epoch using our training data
def train(net, device, loader, optimizer, Loss_fun):
    
    #initialise counters
    epoch_loss = 0
    epoch_acc = 0
    
    #Set Network in train mode
    net.train()

    for i, (x, y) in enumerate(loader):
        
        #load images and labels to device
        x = x.to(device) # x is the image
        y = y.squeeze(1).to(device) # y is the corresponding label, squeeze the channel part as required to train semantic segmentation
        
        #Forward pass of image through network and get output
        fx = net(x)['out']
        
        #Calculate loss using loss function
        loss = Loss_fun(fx, y.long())
        

        # Zero Gradents
        optimizer.zero_grad()
        # Backpropagate Gradents
        loss.backward()
        # Do a single optimization step
        optimizer.step()
        
        #create the cumulative sum of the loss and acc
        epoch_loss += loss.item()
        clear_output(True)
        print("TRAINING: | Itteration [%d/%d] | Loss %.2f |" %(i+1 ,len(loader) , loss.item()))

    #return the avaerage loss and acc from the epoch as well as the logger array       
    return epoch_loss / len(loader)

In [ ]:
#This function should perform a single evaluation epoch and will be passed our validation or evaluation/test data
#it WILL NOT be used to train out model
def evaluate(net, device, loader, Loss_fun):
    
    epoch_loss = 0
    epoch_acc = 0
    
    #Set network in evaluation mode
    #Layers like Dropout will be disabled
    #Layers like Batchnorm will stop calculating running mean and standard deviation
    #and use current stored values
    net.eval()
    
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            
            #load images and labels to device
            x = x.to(device)
            y = y.squeeze(1).to(device) # y is the corresponding label, squeeze the channel part as required to train semantic segmentation
            
            #Forward pass of image through network
            fx = net(x)['out']

            #Calculate loss using loss function
            loss = Loss_fun(fx, y.long())
            
            #log the cumulative sum of the loss and acc
            epoch_loss += loss.item()
            clear_output(True)
            print("EVALUATION: | Itteration [%d/%d] | Loss %.2f |" %(i+1 ,len(loader) , loss.item()))
    
    #return the avaerage loss and acc from the epoch as well as the logger array       
    return epoch_loss / len(loader)

In [ ]:
# best validation loss = 0.156
training_acc_logger = []
validation_acc_logger = []
testing_acc_logger = []
training_loss_logger = []
validation_loss_logger = []
lowest_valid_loss=10
start_time=time.time()
for epoch in range(num_epochs):

    # call the training function and pass training dataloader etc
    train_loss = train(net, device, trainloader, optimizer, Loss_fun)
    training_loss_logger.append(train_loss)
    
    # call the evaluate function and pass validation dataloader etc
    valid_loss = evaluate(net, device, validloader, Loss_fun)
    validation_loss_logger.append(valid_loss)
    
    # If this model has the highest performace on the validation set 
    # then save a checkpoint
    if (valid_loss < lowest_valid_loss):
        lowest_valid_loss = valid_loss
        print("Saving Model")
        torch.save({
            'epoch':                 epoch,
            'model_state_dict':      net.state_dict(),
            'optimizer_state_dict':  optimizer.state_dict(), 
        }, Save_Path)
        
    
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f} |')

print("--- run time: %s seconds ---" % (time.time() - start_time))

In [ ]:
# Plotting the loss and accuracy

plt.figure(figsize = (15,10))
# plt.subplot(1,2,1)
train_x = np.linspace(0, num_epochs, len(training_loss_logger))
plt.plot(train_x, training_loss_logger)
valid_x = np.linspace(0, num_epochs, len(validation_loss_logger))
plt.plot(valid_x, validation_loss_logger)
plt.legend(["Training Loss", "Validation Loss"])
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
#Load Checkpoint
#Check if checkpoint exists

if os.path.isfile(Save_Path):
    #load Checkpoint
    check_point = torch.load(Save_Path)
    net.load_state_dict(check_point['model_state_dict'])
    optimizer.load_state_dict(check_point['optimizer_state_dict'])
    start_epoch = check_point['epoch']
#     best_valid_acc = check_point['valid_acc']
    print("Checkpoint loaded, starting from epoch:", start_epoch+1)